In [154]:
user_name = "parasyte" # REPLACE THIS WITH YOUR HOST MACHINE USER NAME 
root_dir = f"/tmp/finn_dev_{user_name}"
print(root_dir)
# get onnx model from the last NOTEBOOK
filename = root_dir + "/ready_finn.onnx"

/tmp/finn_dev_parasyte


In [155]:
from finn.util.visualization import showSrc, showInNetron


In [156]:
# showInNetron(filename)

In [157]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.core.modelwrapper import ModelWrapper

model = ModelWrapper(filename)

# TIDY UP
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(root_dir + "/tidy.onnx")

showSrc(InferShapes)

class InferShapes(Transformation):
    """Ensure every tensor in the model has a specified shape (ValueInfo)."""

    def apply(self, model):
        # hide your riches!
        hidden_ops = _hide_finn_ops(model)
        # call regular ONNX shape inference
        model = ModelWrapper(si.infer_shapes(model.model))
        # bring back hidden ops
        _restore_finn_ops(model, hidden_ops)
        return (model, False)



In [158]:
# showInNetron(root_dir + "/tidy.onnx")

In [159]:

from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
import torch
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# PRE PROC : NONE
model = ModelWrapper(root_dir + "/tidy.onnx")

#=========================================================
# Here you could add some proprocessing if you need some
#=========================================================

# add input annotation: UINT8 is what we will feed the model during inference
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])


model.save(root_dir + "/full_preproc.onnx")


In [160]:
# showInNetron(root_dir + "/full_preproc.onnx")

In [161]:

from qonnx.transformation.insert_topk import InsertTopK

# POST PROC

# insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = root_dir + "/pre_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

# showInNetron(chkpt_name)

In [162]:

from finn.transformation.streamline import Streamline
# we can see the list of apllied transformations here : showSrc(Streamline)
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper(root_dir + "/pre_post.onnx")
# STREAMLINE
model = model.transform(Streamline())
model.save(root_dir + "/streamlined.onnx")
# showInNetron(root_dir + "/streamlined.onnx")

In [163]:
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper(root_dir + "/streamlined.onnx")

# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(RoundAndClipThresholds())

# bit of tidy-up
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

model.save(root_dir + "/streamlined_merged_and_ready.onnx")
# showInNetron(root_dir + "/streamlined_merged_and_ready.onnx")

In [164]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from qonnx.core.modelwrapper import ModelWrapper

# TO HW LAYERS

model = ModelWrapper(root_dir + "/streamlined_merged_and_ready.onnx")
model = model.transform(to_hw.InferLabelSelectLayer())
model = model.transform(to_hw.InferChannelwiseLinearLayer())
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
model = model.transform(to_hw.InferThresholdingLayer())
model.save(root_dir + "/hw.onnx")
# showInNetron(root_dir + "/hw.onnx")

In [165]:
from qonnx.core.modelwrapper import ModelWrapper

hw_model = ModelWrapper(root_dir + "/hw.onnx")

# listar todos los tipos de nodos que contiene
op_types = set([n.op_type for n in hw_model.graph.node])
print("Tipos de nodos encontrados en hw.onnx:\n", op_types)



Tipos de nodos encontrados en hw.onnx:
 {'Reshape', 'Mul', 'MVAU', 'TopK', 'Add'}


In [166]:
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper(root_dir + "/hw.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(root_dir + "/df_part.onnx")
# showInNetron(root_dir + "/df_part.onnx")

# We see that our entire model is now part of a child model in "StreamingDataflowPartition"

In [167]:
from qonnx.custom_op.registry import getCustomOp
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# showInNetron(dataflow_model_filename)

In [168]:
# print the names of the supported PYNQ boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z2"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 20

dict_keys(['Ultra96', 'Ultra96-V2', 'Pynq-Z1', 'Pynq-Z2', 'ZCU102', 'ZCU104', 'ZCU111', 'RFSoC2x2', 'RFSoC4x2', 'KV260_SOM'])


In [169]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
model = ModelWrapper(dataflow_model_filename)

model = model.transform(SpecializeLayers(fpga_part))

showSrc(SpecializeLayers)

model.save(root_dir + "/to_hw_conv.onnx")
# showInNetron(root_dir + "/to_hw_conv.onnx")

class SpecializeLayers(Transformation):
    """Specialize all layers to either HLS or RTL variants"""

    def __init__(self, fpgapart):
        super().__init__()
        self.fpgapart = fpgapart

    def apply(self, model):
        graph = model.graph
        node_ind = 0
        graph_modified = False
        for node in graph.node:
            # Skip nodes that are not hw layers
            if not node.domain == "finn.custom_op.fpgadataflow":
                continue
            node_ind += 1
            impl_style = _determine_impl_style(node, self.fpgapart, model)
            optype = node.op_type + "_" + impl_style

            new_node = helper.make_node(
                optype,
                node.input,
                node.output,
                domain="finn.custom_op.fpgadataflow." + impl_style,
            )
            # add all attributes
            for attribute in node.attribute:
                if attribute.name != "preferred_impl_style":
                    new_no

In [170]:
model = ModelWrapper(root_dir + "/to_hw_conv.onnx")

fc0 = model.graph.node[0] #1st MVAU
fc0w = getCustomOp(fc0)

print("CustomOp wrapper is of class " + fc0w.__class__.__name__)

fc0w.get_nodeattr_types()

CustomOp wrapper is of class MVAU_hls


{'PE': ('i', True, 0),
 'SIMD': ('i', True, 0),
 'MW': ('i', True, 0),
 'MH': ('i', True, 0),
 'resType': ('s', False, 'auto', {'auto', 'dsp', 'lut'}),
 'ActVal': ('i', False, 0),
 'inputDataType': ('s', True, ''),
 'weightDataType': ('s', True, ''),
 'outputDataType': ('s', True, ''),
 'accDataType': ('s', False, 'INT32'),
 'binaryXnorMode': ('i', False, 0, {0, 1}),
 'noActivation': ('i', False, 0, {0, 1}),
 'numInputVectors': ('ints', False, [1]),
 'mem_mode': ('s',
  False,
  'internal_decoupled',
  {'external', 'internal_decoupled', 'internal_embedded'}),
 'ram_style': ('s', False, 'auto', {'auto', 'block', 'distributed', 'ultra'}),
 'ram_style_thresholds': ('s',
  False,
  'auto',
  {'auto', 'block', 'distributed'}),
 'runtime_writeable_weights': ('i', False, 0, {0, 1}),
 'backend': ('s', True, 'fpgadataflow'),
 'preferred_impl_style': ('s', False, '', {'', 'hls', 'rtl'}),
 'code_gen_dir_ipgen': ('s', False, ''),
 'ipgen_path': ('s', False, ''),
 'ip_path': ('s', False, ''),
 'ip_

In [171]:

fc0 = model.graph.node[0] #1st MVAU
fc0w = getCustomOp(fc0)
fc0w.set_nodeattr("SIMD",16)
fc0w.set_nodeattr("PE",8)

fc0w = getCustomOp(fc0)
print("SIMD:", fc0w.get_nodeattr("SIMD"))
print("PE:", fc0w.get_nodeattr("PE"))

SIMD: 16
PE: 8


In [182]:
from finn.transformation.fpgadataflow.set_folding import SetFolding

TARGET_FPS = 100
target_clk_cycles_per_frame = 1/(TARGET_FPS * 10e-9)

model = ModelWrapper(root_dir + "/to_hw_conv.onnx")
# Actual method SetFolding that will automatically modify the folding settings
model.transform(SetFolding(target_cycles_per_frame=target_clk_cycles_per_frame, mvau_wwidth_max=64, two_pass_relaxation=False))

fc0 = model.graph.node[0] #1st MVAU
fc0w = getCustomOp(fc0)
fc0w.set_nodeattr("SIMD",3)
fc0w.set_nodeattr("PE",1)

fc0w = getCustomOp(fc0)
print("SIMD:", fc0w.get_nodeattr("SIMD"))
print("PE:", fc0w.get_nodeattr("PE"))
model.save(root_dir + "/to_hw_folded.onnx")
# showInNetron(root_dir + "/to_hw_folded.onnx")

SIMD: 3
PE: 1


In [174]:
# ESTIMATE
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = root_dir + "/to_hw_folded.onnx"

estimates_output_dir = "output_estimates_only"

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 64,
    target_fps          = 100,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

build.build_dataflow_cfg(model_file, cfg_estimates)

Running step: step_qonnx_to_finn [1/10]
Running step: step_tidy_up [2/10]
Running step: step_streamline [3/10]


Previous run results deleted!
Building dataflow accelerator from /tmp/finn_dev_parasyte/to_hw_folded.onnx
Intermediate outputs will be generated in /tmp/finn_dev_parasyte
Final outputs will be generated in output_estimates_only
Build log is at output_estimates_only/build_dataflow.log
Running step: step_qonnx_to_finn [1/10]
Running step: step_tidy_up [2/10]
Running step: step_streamline [3/10]


Running step: step_convert_to_hw [4/10]
Running step: step_create_dataflow_partition [5/10]
Running step: step_specialize_layers [6/10]
Running step: step_target_fps_parallelization [7/10]
Running step: step_apply_folding_config [8/10]
Running step: step_minimize_bit_width [9/10]
Running step: step_generate_estimate_reports [10/10]


Running step: step_convert_to_hw [4/10]
Running step: step_create_dataflow_partition [5/10]
Running step: step_specialize_layers [6/10]
Running step: step_target_fps_parallelization [7/10]
Running step: step_apply_folding_config [8/10]
Running step: step_minimize_bit_width [9/10]
Running step: step_generate_estimate_reports [10/10]
Completed successfully


0

In [175]:
! cat {estimates_output_dir}/report/estimate_network_performance.json


{
  "critical_path_cycles": 403520,
  "max_cycles": 393216,
  "max_cycles_node_name": "MVAU_hls_0",
  "estimated_throughput_fps": 254.31315104166666,
  "estimated_latency_ns": 4035200.0
}

In [176]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

{'MVAU_hls_0': 393216,
 'MVAU_hls_1': 8192,
 'MVAU_hls_2': 2048,
 'MVAU_rtl_0': 64}

In [177]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")


{'MVAU_hls_0': {'BRAM_18K': 96,
  'BRAM_efficiency': 0.8888888888888888,
  'LUT': 375,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_hls_1': {'BRAM_18K': 4,
  'BRAM_efficiency': 0.8888888888888888,
  'LUT': 371,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_hls_2': {'BRAM_18K': 1,
  'BRAM_efficiency': 0.8888888888888888,
  'LUT': 370,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MVAU_rtl_0': {'BRAM_18K': 1,
  'BRAM_efficiency': 0.027777777777777776,
  'LUT': 0,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 1},
 'total': {'BRAM_18K': 102.0, 'LUT': 1116.0, 'URAM': 0.0, 'DSP': 1.0}}

In [178]:
# Actual hardware build, ZYNQ BUILD
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild

model = ModelWrapper(root_dir + "/to_hw_folded.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns,partition_model_dir="./test",enable_debug=True))

/home/parasyte/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 6 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/home/parasyte/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:234: UserWarning: Input FIFO for IODMA_hls_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/parasyte/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:294: UserWarning: Output FIFO for MVAU_rtl_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/parasyte/finn/src/finn/transformation/fpgadataflow/create_stitched_ip.py:290: UserWarning: First node is not StreamingFIFO or IODMA.
                You may experience incorrect stitched-IP rtlsim or hardware
                behavior. It is strongly recommended to insert FIFOs prior to
               

In [179]:
model.save( "post_synth.onnx")

In [183]:
showInNetron("post_synth.onnx")


Serving 'post_synth.onnx' at http://0.0.0.0:8081


In [185]:
model = ModelWrapper("post_synth.onnx")

In [186]:
model.model.metadata_props

[key: "floorplan_json"
value: "/tmp/finn_dev_parasyte/vitis_floorplan_gskc26cu/floorplan.json"
, key: "vivado_pynq_proj"
value: "/tmp/finn_dev_parasyte/vivado_zynq_proj_icavrp1d"
, key: "bitfile"
value: "/tmp/finn_dev_parasyte/vivado_zynq_proj_icavrp1d/resizer.bit"
, key: "hw_handoff"
value: "/tmp/finn_dev_parasyte/vivado_zynq_proj_icavrp1d/resizer.hwh"
, key: "vivado_synth_rpt"
value: "/tmp/finn_dev_parasyte/vivado_zynq_proj_icavrp1d/synth_report.xml"
, key: "platform"
value: "zynq-iodma"
]